In [15]:
%run config

#to see if scope is there

#dbutils.secrets.listScopes()
client_id = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientid") 

client_secret = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientsecret") 

new_refresh_token = dbutils.secrets.get(scope = "key_vault_secrets", key = "newrefreshtoken")

activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "activityidpath") 

historical_activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "historicalactivitydfpath") 

segment_effort_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmenteffortpath") 

import requests

import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



auth_url = "https://www.strava.com/oauth/token"

activites_url = "https://www.strava.com/api/v3/athlete/activities"





payload = {

    'client_id':  client_id,

    'client_secret': client_secret,

    'refresh_token': new_refresh_token,

    'grant_type': 'refr

### API Query to get more specific details for each activity, need to pass each activity off individually 

In [16]:
#get full activity dataset from what is written in storage, should be all activites
full_activity_dataset = spark.read.format("delta").load(historical_activity_id_path)

In [17]:
full_activity_dataset.show()

+------------+--------------------+-------------------+--------+-----------+------------+----------+--------------------+--------------------+--------------------+
|activity_ids|          start_date|      activity_name|distance|moving_time|elapsed_time|sport_type|total_elevation_gain|    ingest_file_name|         ingested_at|
+------------+--------------------+-------------------+--------+-----------+------------+----------+--------------------+--------------------+--------------------+
|  7480841356|2022-07-16T22:26:16Z|          city wide| 32228.3|       5488|        6756|      Ride|               375.5|activity_information|2022-09-19 21:37:...|
|  7454211244|2022-07-12T01:28:02Z|       straight up |  1710.1|       1297|        1315|      Walk|               142.8|activity_information|2022-09-19 21:37:...|
|  7437771392|2022-07-09T00:06:28Z|        friday ride| 23022.7|       4455|        5896|      Ride|               296.4|activity_information|2022-09-19 21:37:...|
|  7411376407|20

In [19]:
#grab all of the disinct activity IDs 
full_activity_ids = full_activity_dataset.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

In [20]:
subset = full_activity_ids[:25]

In [41]:
subset

Out[28]: [5508615942,
 5972584497,
 5740260874,
 4838912326,
 3809768344,
 4079983225,
 6129061619,
 5718648252,
 5987842682,
 6040654473,
 3865986737,
 3764074685,
 3871052555,
 4768097716,
 4414021787,
 5997390802,
 6114231607,
 6059385275,
 5524575370,
 3876332481,
 4730507502,
 5755543494,
 4382047703,
 4910436714,
 7743374567]

In [21]:
#Dependencies 
from pyspark.sql.functions import * 
import pandas as pd

In [22]:
#Move activity_id_list and segment_id_list out of the for loop
def query_segments(activity_ids):
    """Gets all segment_ids for each activity_id submitted"""
    df = pd.DataFrame()
    activity_id_list =[]
    segment_id_list =[]
    for id in activity_ids:
        activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",id)
        header = {'Authorization': 'Bearer ' + access_token}
        param = {'per_page': 200, 'page': 1}
        my_activity = requests.get(activity_id_urls, headers=header, params=param).json()

        segment_effort_count =  len(my_activity['segment_efforts'])
        count = 0
        while count < segment_effort_count:

            activity_id = my_activity['segment_efforts'][count]['activity']['id']
            segment_id = my_activity['segment_efforts'][count]['id']
            activity_id_list.append(activity_id)
            segment_id_list.append(segment_id)
                  
            columns = ['segment_id', 'activity_id']
            extracted_data = [segment_id_list, activity_id_list]
            segment_df = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))

            df= df.append(segment_df)
            count += 1

    #convert pandas df to spark
        
    segment_spark_df = spark.createDataFrame(df)

    segment_spark_df = segment_spark_df.withColumn("ingest_file_name", lit("segment_efforts_ids")) \
                                .withColumn("ingested_at", lit(current_timestamp()))

    return segment_spark_df







In [31]:
activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",4414021787)
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_activity = requests.get(activity_id_urls, headers=header, params=param).json()

In [35]:
len(my_activity['segment_efforts'])

Out[22]: 1

In [38]:
test_ids = [5508615942,4414021787]
test_2 = query_segments(test_ids)
test_2.show()

+-------------------+-----------+-------------------+--------------------+
|         segment_id|activity_id|   ingest_file_name|         ingested_at|
+-------------------+-----------+-------------------+--------------------+
|2842224702954201064| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702954201064| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702952120296| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702954201064| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702952120296| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702953071592| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702954201064| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702952120296| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702953071592| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702951809000| 5508615942|segment_efforts_ids|2022-11-14 22:54:...|
|2842224702954201064| 550

In [40]:
#got stuck again on 1-0
test_2.groupBy('activity_id').agg(countDistinct('segment_id')).show()

+-----------+-----------------+
|activity_id|count(segment_id)|
+-----------+-----------------+
| 5508615942|              108|
| 4414021787|                1|
+-----------+-----------------+

In [36]:
subset_Segment_df.show()

+-------------------+-----------+-------------------+--------------------+
|         segment_id|activity_id|   ingest_file_name|         ingested_at|
+-------------------+-----------+-------------------+--------------------+
|2842224702954201064| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702954201064| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702952120296| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702954201064| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702952120296| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702953071592| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702954201064| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702952120296| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702953071592| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702951809000| 5508615942|segment_efforts_ids|2022-11-14 22:52:...|
|2842224702954201064| 550

In [23]:
subset_Segment_df = query_segments(subset)

In [29]:
subset_Segment_df.filter(subset_Segment_df.activity_id == 4414021787).show()

+-------------------+-----------+-------------------+--------------------+
|         segment_id|activity_id|   ingest_file_name|         ingested_at|
+-------------------+-----------+-------------------+--------------------+
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 4414021787|segment_efforts_ids|2022-11-14 22:36:...|
|2768652401740334924| 441

In [30]:
subset_Segment_df.groupBy('activity_id').agg(countDistinct('segment_id')).show()

+-----------+-----------------+
|activity_id|count(segment_id)|
+-----------+-----------------+
| 5508615942|              108|
| 5972584497|                3|
| 7743374567|                1|
| 5740260874|                3|
| 4838912326|               12|
| 6129061619|               34|
| 5718648252|               21|
| 5987842682|               20|
| 6040654473|               25|
| 3865986737|              128|
| 3871052555|               14|
| 4768097716|                2|
| 4414021787|                1|
| 5997390802|                2|
| 6059385275|               25|
| 6114231607|               19|
| 5524575370|               32|
| 3876332481|              132|
| 4730507502|               10|
| 5755543494|               69|
+-----------+-----------------+

In [ ]:
#query all segments from all activities 
segment_id_df = query_segments(full_activity_ids)

In [ ]:
write_dataframe_to_storage(segment_id_df,segment_effort_path, "overwriteSchema","overwrite" )

In [ ]:
segments_in_storage = spark.read.format("delta").load(segment_effort_path)

In [ ]:
#hopefull.groupBy('activity_id').agg(countDistinct('segment_id')).show()

In [ ]:
activity_list = hopefull.select('activity_id').distinct().collect()
len(activity_list)
len(subset)

In [ ]:
#need to get distinct activity_ids and run them through the segment
#we make 1 api query in the first script, so gonna be allowed 99 with this run 

In [ ]:
#get all of the activity_ids, limit to 15 as thats how many we can run in a single query
#also need to query the activities we have written to segment storage, so as not to repeat 
#Strava API usage is limited on a per-application basis using both a 15-minute and daily request limit. The default rate limit allows 100 requests every 15 minutes, 
# with up to 1,000 requests per day.

#compare current activites vs what is written, 
#going to need to write some try and excepts for expecting these values in return 